In [2]:
from pyspark.sql import SparkSession
import os 

# Spark + Iceberg 설정
spark = SparkSession.builder \
    .appName("IcebergBlogDemo") \
    .config("spark.jars", "/home/jovyan/iceberg-spark.jar") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.demo.type", "hadoop") \
    .config("spark.sql.catalog.demo.warehouse", "file:///home/jovyan/data/warehouse") \
    .getOrCreate()

print("✅ Spark + Iceberg 설정 완료!")

# 카탈로그 확인
spark.sql("SHOW CATALOGS").show()

✅ Spark + Iceberg 설정 완료!
+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [3]:
# 1. 새로운 MOR 테이블 생성
print("🔄 새로운 MOR 테이블 생성 중...")

# 기존 테이블이 있다면 삭제
try:
    spark.sql("DROP TABLE IF EXISTS demo.blog.orders_mor")
    print("✅ 기존 MOR 테이블 삭제 완료")
except:
    pass

# MOR 설정으로 새 테이블 생성
spark.sql("""
    CREATE TABLE demo.blog.orders_mor (
        order_id BIGINT,
        customer_id BIGINT,
        product_name STRING,
        order_date DATE,
        amount DECIMAL(10,2),
        status STRING
    ) USING ICEBERG
    PARTITIONED BY (order_date)
    TBLPROPERTIES (
        'write.delete.mode' = 'merge-on-read',
        'write.update.mode' = 'merge-on-read',
        'write.merge.mode' = 'merge-on-read'
    )
""")

print("✅ MOR 테이블 생성 완료!")

# 테이블 설정 확인
print("\n📊 MOR 테이블 설정:")
mor_props = spark.sql("SHOW TBLPROPERTIES demo.blog.orders_mor").collect()
for prop in mor_props:
    if any(keyword in prop['key'] for keyword in ['write', 'delete', 'merge']):
        print(f"  {prop['key']}: {prop['value']}")

🔄 새로운 MOR 테이블 생성 중...
✅ 기존 MOR 테이블 삭제 완료
✅ MOR 테이블 생성 완료!

📊 MOR 테이블 설정:
  write.delete.mode: merge-on-read
  write.merge.mode: merge-on-read
  write.parquet.compression-codec: zstd
  write.update.mode: merge-on-read


In [4]:
# 2. 100개 랜덤 데이터 생성 (이전과 동일한 함수 사용)
import pandas as pd
import random
from datetime import datetime, timedelta
from pyspark.sql.functions import col

def generate_sample_data(num_records=100):
    # [이전과 동일한 함수 내용]
    # 상품 목록, 가격, 상태 등등...
    
    # 상품 목록
    products = [
        'iPhone 14', 'iPhone 14 Pro', 'MacBook Pro', 'MacBook Air', 'iPad Air', 
        'iPad Pro', 'AirPods', 'AirPods Pro', 'Apple Watch', 'Magic Mouse',
        'Magic Keyboard', 'iMac', 'Mac Studio', 'Apple TV', 'HomePod',
        'iPhone 13', 'iPhone SE', 'Mac Mini', 'Studio Display', 'Pro Display'
    ]
    
    prices = {
        'iPhone 14': 999.99, 'iPhone 14 Pro': 1199.99, 'MacBook Pro': 1999.99,
        'MacBook Air': 1299.99, 'iPad Air': 649.99, 'iPad Pro': 899.99,
        'AirPods': 199.99, 'AirPods Pro': 299.99, 'Apple Watch': 399.99,
        'Magic Mouse': 99.99, 'Magic Keyboard': 199.99, 'iMac': 1499.99,
        'Mac Studio': 2199.99, 'Apple TV': 149.99, 'HomePod': 349.99,
        'iPhone 13': 799.99, 'iPhone SE': 449.99, 'Mac Mini': 799.99,
        'Studio Display': 1799.99, 'Pro Display': 5999.99
    }
    
    statuses = ['completed', 'pending', 'shipped', 'cancelled', 'processing']
    
    # 날짜 범위 (3개월)
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 3, 31)
    
    data = []
    for i in range(1, num_records + 1):
        random_days = random.randint(0, (end_date - start_date).days)
        order_date = start_date + timedelta(days=random_days)
        
        product = random.choice(products)
        base_price = prices[product]
        price_variation = random.uniform(0.8, 1.2)
        final_price = round(base_price * price_variation, 2)
        
        data.append({
            'order_id': i,
            'customer_id': random.randint(100, 999),
            'product_name': product,
            'order_date': order_date.strftime('%Y-%m-%d'),
            'amount': final_price,
            'status': random.choice(statuses)
        })
    
    return data

# 100개 데이터 생성
print("🎲 MOR 테이블용 랜덤 데이터 100개 생성 중...")
mor_data = generate_sample_data(100)

# DataFrame 생성 및 삽입
df_pandas = pd.DataFrame(mor_data)
df_spark = spark.createDataFrame(df_pandas)
df_spark = df_spark.withColumn("order_date", col("order_date").cast("date"))

df_spark.write \
    .format("iceberg") \
    .mode("append") \
    .saveAsTable("demo.blog.orders_mor")

print("✅ MOR 테이블에 100개 데이터 삽입 완료!")

# 삽입 결과 확인
spark.sql("SELECT COUNT(*) as total_records FROM demo.blog.orders_mor").show()

🎲 MOR 테이블용 랜덤 데이터 100개 생성 중...
✅ MOR 테이블에 100개 데이터 삽입 완료!
+-------------+
|total_records|
+-------------+
|          100|
+-------------+



In [5]:
# 3. 다양한 변경 작업으로 Delete Files 생성
print("🔄 다양한 변경 작업으로 Delete Files 생성 중...")

mor_table_path = "/home/jovyan/data/warehouse/blog/orders_mor"

# 변경 전 파일 상태 확인
import glob
before_files = glob.glob(f"{mor_table_path}/**/*.parquet", recursive=True)
print(f"변경 전 파일 수: {len(before_files)}")

# 1차: 일부 데이터 삭제
print("\n🗑️ 1차: 일부 주문 삭제 (order_id <= 20)")
spark.sql("DELETE FROM demo.blog.orders_mor WHERE order_id <= 20")

# 2차: 상태 업데이트 (UPDATE는 내부적으로 DELETE + INSERT)
print("\n📝 2차: 주문 상태 업데이트")
spark.sql("""
    UPDATE demo.blog.orders_mor 
    SET status = 'cancelled' 
    WHERE order_id BETWEEN 21 AND 30
""")

# 3차: 추가 삭제 (특정 상품)
print("\n🗑️ 3차: 특정 상품 주문 삭제")
spark.sql("""
    DELETE FROM demo.blog.orders_mor 
    WHERE product_name IN ('iPhone SE', 'Magic Mouse')
""")

# 4차: 새로운 데이터 추가 (대조를 위해)
print("\n➕ 4차: 새로운 주문 추가")
spark.sql("""
    INSERT INTO demo.blog.orders_mor VALUES 
    (999, 888, 'New Product', DATE('2023-04-01'), 599.99, 'new'),
    (998, 777, 'Another Product', DATE('2023-04-02'), 299.99, 'new')
""")

print("✅ 모든 변경 작업 완료!")

# 최종 결과 확인
final_count = spark.sql("SELECT COUNT(*) FROM demo.blog.orders_mor").collect()[0][0]
print(f"최종 레코드 수: {final_count}")

🔄 다양한 변경 작업으로 Delete Files 생성 중...
변경 전 파일 수: 60

🗑️ 1차: 일부 주문 삭제 (order_id <= 20)

📝 2차: 주문 상태 업데이트

🗑️ 3차: 특정 상품 주문 삭제

➕ 4차: 새로운 주문 추가
✅ 모든 변경 작업 완료!
최종 레코드 수: 78


In [8]:
# 4. Delete Files 생성 확인
print("🔍 Delete Files 생성 확인:")
print("=" * 40)

# 모든 파일 재스캔
after_files = glob.glob(f"{mor_table_path}/**/*.parquet", recursive=True)
delete_files = [f for f in after_files if 'delete' in os.path.basename(f).lower()]
data_files = [f for f in after_files if 'delete' not in os.path.basename(f).lower()]

print(f"📊 총 파일 수: {len(after_files)}")
print(f"📄 데이터 파일: {len(data_files)}개")
print(f"🗑️ Delete 파일: {len(delete_files)}개")

if delete_files:
    print("\n🎉 Delete Files 생성 성공!")
    for i, delete_file in enumerate(delete_files):
        file_size = os.path.getsize(delete_file)
        relative_path = delete_file.replace(mor_table_path, "")
        print(f"  {i+1}. {relative_path} ({file_size:,} bytes)")
        
        # Delete 파일 내용 확인
        try:
            delete_df = spark.read.parquet(delete_file)
            delete_count = delete_df.count()
            print(f"     삭제 대상 레코드: {delete_count}개")
            
            if delete_count > 0 and delete_count <= 10:
                print("     삭제 대상 내용:")
                delete_df.show(truncate=False)
        except Exception as e:
            print(f"     읽기 실패: {e}")
else:
    print("❌ Delete Files가 생성되지 않았습니다.")
    print("다른 방법을 시도해보겠습니다...")

🔍 Delete Files 생성 확인:
📊 총 파일 수: 101
📄 데이터 파일: 72개
🗑️ Delete 파일: 29개

🎉 Delete Files 생성 성공!
  1. /data/order_date=2023-01-03/00000-20-e1625583-dde6-47ea-bd2c-cdeb07d86e69-00001-deletes.parquet (1,595 bytes)
     삭제 대상 레코드: 1개
     삭제 대상 내용:
+--------------------------------------------------------------------------------------------------------------------------------------+---+
|file_path                                                                                                                             |pos|
+--------------------------------------------------------------------------------------------------------------------------------------+---+
|file:/home/jovyan/data/warehouse/blog/orders_mor/data/order_date=2023-01-03/00000-5-226e50bc-6f72-465e-be45-604bea069c98-00003.parquet|0  |
+--------------------------------------------------------------------------------------------------------------------------------------+---+

  2. /data/order_date=2023-01-12/00000-20-e1625583-dde

In [9]:
# 5. MOR 테이블의 완전한 파일 구조 분석
print("📁 MOR 테이블 완전한 파일 구조 분석:")
print("=" * 50)

def analyze_mor_structure(path, prefix=""):
    items = []
    try:
        for item in sorted(os.listdir(path)):
            if item.startswith('.'):
                continue
            item_path = os.path.join(path, item)
            
            if os.path.isfile(item_path):
                size = os.path.getsize(item_path)
                
                # 파일 타입 분류
                if item.endswith('.parquet'):
                    if 'delete' in item.lower():
                        file_type = "🗑️ DELETE FILE"
                    else:
                        file_type = "📄 DATA FILE"
                elif item.endswith('.avro'):
                    if item.startswith('snap-'):
                        file_type = "📋 MANIFEST LIST"
                    else:
                        file_type = "📄 MANIFEST FILE"
                elif item.endswith('.json'):
                    file_type = "📊 METADATA FILE"
                else:
                    file_type = "📄 OTHER FILE"
                
                items.append(f"{prefix}├── {item} ({size:,} bytes) {file_type}")
            elif os.path.isdir(item_path):
                items.append(f"{prefix}├── 📁 {item}/")
                items.extend(analyze_mor_structure(item_path, prefix + "│   "))
    except:
        pass
    return items

structure = analyze_mor_structure(mor_table_path)
for item in structure:
    print(item)

📁 MOR 테이블 완전한 파일 구조 분석:
├── 📁 data/
│   ├── 📁 order_date=2023-01-01/
│   │   ├── 00000-5-226e50bc-6f72-465e-be45-604bea069c98-00001.parquet (1,834 bytes) 📄 DATA FILE
│   ├── 📁 order_date=2023-01-02/
│   │   ├── 00000-5-226e50bc-6f72-465e-be45-604bea069c98-00002.parquet (1,865 bytes) 📄 DATA FILE
│   ├── 📁 order_date=2023-01-03/
│   │   ├── 00000-20-e1625583-dde6-47ea-bd2c-cdeb07d86e69-00001-deletes.parquet (1,595 bytes) 🗑️ DELETE FILE
│   │   ├── 00000-20-e1625583-dde6-47ea-bd2c-cdeb07d86e69-00001.parquet (1,806 bytes) 📄 DATA FILE
│   │   ├── 00000-5-226e50bc-6f72-465e-be45-604bea069c98-00003.parquet (1,814 bytes) 📄 DATA FILE
│   ├── 📁 order_date=2023-01-05/
│   │   ├── 00000-5-226e50bc-6f72-465e-be45-604bea069c98-00004.parquet (1,835 bytes) 📄 DATA FILE
│   ├── 📁 order_date=2023-01-06/
│   │   ├── 00000-5-226e50bc-6f72-465e-be45-604bea069c98-00005.parquet (1,813 bytes) 📄 DATA FILE
│   ├── 📁 order_date=2023-01-07/
│   │   ├── 00000-5-226e50bc-6f72-465e-be45-604bea069c98-00006.parquet (1,

In [11]:
# =============================================================================
# 🔄 MOR(Merge-on-Read) 방식 Iceberg 테이블 완전 분석
# =============================================================================

print("🎉 MOR 테이블 생성 완료! 다음 단계들:")
print("=" * 50)
print("📚 Step 6: Data Layer 심화 분석 (Delete Files 포함)")
print("📚 Step 7: Metadata Layer 심화 분석") 
print("📚 Step 8: Time Travel 실습")
print("📚 Step 9: COW vs MOR 비교 분석")
print("📚 Step 10: Schema Evolution 실습")
print("📚 Step 11: Compaction 실습")
print("📚 Step 12: 성능 벤치마크")

🎉 MOR 테이블 생성 완료! 다음 단계들:
📚 Step 6: Data Layer 심화 분석 (Delete Files 포함)
📚 Step 7: Metadata Layer 심화 분석
📚 Step 8: Time Travel 실습
📚 Step 9: COW vs MOR 비교 분석
📚 Step 10: Schema Evolution 실습
📚 Step 11: Compaction 실습
📚 Step 12: 성능 벤치마크


In [12]:
# =============================================================================
# 📁 MOR Data Layer 심화 분석 - Delete Files 중심
# =============================================================================

print("🔍 MOR 테이블의 Data Layer 완전 분석")
print("=" * 50)

mor_table_path = "/home/jovyan/data/warehouse/blog/orders_mor"

# 1. 전체 파일 분류
import glob
import os

all_files = glob.glob(f"{mor_table_path}/**/*.parquet", recursive=True)
data_files = [f for f in all_files if 'delete' not in os.path.basename(f).lower()]
delete_files = [f for f in all_files if 'delete' in os.path.basename(f).lower()]

print(f"📊 파일 현황:")
print(f"  총 Parquet 파일: {len(all_files)}개")
print(f"  📄 데이터 파일: {len(data_files)}개")
print(f"  🗑️ Delete 파일: {len(delete_files)}개")

# 2. Delete Files 상세 분석
print(f"\n🗑️ Delete Files 상세 분석:")
print("=" * 30)

for i, delete_file in enumerate(delete_files):
    file_size = os.path.getsize(delete_file)
    relative_path = delete_file.replace(mor_table_path, "")
    
    print(f"\n📄 Delete File #{i+1}: {relative_path}")
    print(f"  크기: {file_size:,} bytes")
    
    try:
        # Delete 파일 내용 읽기
        delete_df = spark.read.parquet(delete_file)
        delete_count = delete_df.count()
        
        print(f"  삭제 레코드 수: {delete_count}개")
        print("  스키마:")
        delete_df.printSchema()
        
        if delete_count > 0:
            print("  삭제 대상 데이터:")
            delete_df.show(10, truncate=False)
            
            # Delete file 타입 확인 (Positional vs Equality)
            columns = delete_df.columns
            if 'file_path' in columns and 'pos' in columns:
                print("  📍 타입: Positional Delete File")
                print("  👉 특정 파일의 특정 행 위치로 삭제")
            else:
                print("  🟰 타입: Equality Delete File") 
                print("  👉 컬럼 값으로 삭제")
                
    except Exception as e:
        print(f"  ❌ 읽기 실패: {e}")

# 3. 데이터 파일별 Delete 관계 분석
print(f"\n📄 데이터 파일과 Delete 파일 관계:")
print("=" * 40)

for i, data_file in enumerate(data_files[:5]):  # 처음 5개만
    file_size = os.path.getsize(data_file)
    relative_path = data_file.replace(mor_table_path, "")
    
    print(f"\n📄 데이터 파일 #{i+1}: {relative_path}")
    print(f"  크기: {file_size:,} bytes")
    
    # 데이터 파일 내용 확인
    data_df = spark.read.parquet(data_file)
    record_count = data_df.count()
    
    if record_count > 0:
        # 날짜 범위 확인
        from pyspark.sql.functions import min as spark_min, max as spark_max
        date_range = data_df.agg(
            spark_min("order_date").alias("min_date"),
            spark_max("order_date").alias("max_date")
        ).collect()[0]
        
        print(f"  레코드 수: {record_count}개")
        print(f"  날짜 범위: {date_range['min_date']} ~ {date_range['max_date']}")
        
        # 이 파일에 영향을 주는 Delete Files 확인
        # (실제로는 엔진이 처리하지만, 개념적으로 확인)
        print(f"  🗑️ 관련 Delete Files: {len(delete_files)}개 (전체 적용)")

🔍 MOR 테이블의 Data Layer 완전 분석
📊 파일 현황:
  총 Parquet 파일: 101개
  📄 데이터 파일: 72개
  🗑️ Delete 파일: 29개

🗑️ Delete Files 상세 분석:

📄 Delete File #1: /data/order_date=2023-01-03/00000-20-e1625583-dde6-47ea-bd2c-cdeb07d86e69-00001-deletes.parquet
  크기: 1,595 bytes
  삭제 레코드 수: 1개
  스키마:
root
 |-- file_path: string (nullable = true)
 |-- pos: long (nullable = true)

  삭제 대상 데이터:
+--------------------------------------------------------------------------------------------------------------------------------------+---+
|file_path                                                                                                                             |pos|
+--------------------------------------------------------------------------------------------------------------------------------------+---+
|file:/home/jovyan/data/warehouse/blog/orders_mor/data/order_date=2023-01-03/00000-5-226e50bc-6f72-465e-be45-604bea069c98-00003.parquet|0  |
+---------------------------------------------------------------------

In [14]:
# =============================================================================
# 🗂️ MOR Metadata Layer 심화 분석
# =============================================================================

print("🔍 MOR 테이블의 Metadata Layer 분석")
print("=" * 50)

metadata_path = f"{mor_table_path}/metadata"

# 1. 메타데이터 파일들 분류
metadata_files = {
    'metadata_files': [],
    'manifest_files': [], 
    'manifest_lists': [],
    'other_files': []
}

for file in os.listdir(metadata_path):
    file_path = os.path.join(metadata_path, file)
    size = os.path.getsize(file_path)
    
    if file.endswith('.metadata.json'):
        metadata_files['metadata_files'].append((file, size))
    elif file.endswith('.avro') and 'manifest' in file and not file.startswith('snap-'):
        metadata_files['manifest_files'].append((file, size))
    elif file.startswith('snap-') and file.endswith('.avro'):
        metadata_files['manifest_lists'].append((file, size))
    else:
        metadata_files['other_files'].append((file, size))

print("📊 MOR 테이블 메타데이터 현황:")
for category, files in metadata_files.items():
    print(f"  {category.replace('_', ' ').title()}: {len(files)}개")

# 2. 최신 Metadata 파일 분석
print(f"\n📋 최신 Metadata 파일 분석:")
print("=" * 30)

latest_metadata = max(metadata_files['metadata_files'], key=lambda x: x[1])[0]
metadata_file_path = os.path.join(metadata_path, latest_metadata)

import json
with open(metadata_file_path, 'r') as f:
    metadata = json.load(f)

print(f"📄 파일: {latest_metadata}")
print(f"📊 포맷 버전: {metadata['format-version']}")
print(f"🆔 테이블 UUID: {metadata['table-uuid']}")

# 스냅샷 분석 (MOR 특성상 더 많은 스냅샷)
snapshots = metadata.get('snapshots', [])
print(f"\n📸 스냅샷 분석 ({len(snapshots)}개):")
for i, snapshot in enumerate(snapshots):
    operation = snapshot.get('operation', 'unknown')
    timestamp = snapshot.get('timestamp-ms', 0)
    summary = snapshot.get('summary', {})
    
    print(f"  {i+1}. ID: {snapshot['snapshot-id']}")
    print(f"     작업: {operation}")
    print(f"     시간: {timestamp}")
    
    # MOR 특성 확인
    if 'added-delete-files' in summary:
        print(f"     🗑️ 추가된 Delete 파일: {summary['added-delete-files']}")
    if 'deleted-data-files' in summary:
        print(f"     📄 삭제된 데이터 파일: {summary['deleted-data-files']}")

# 3. Manifest Files에서 Delete Files 추적 확인
print(f"\n📋 Manifest Files에서 Delete Files 추적:")
print("=" * 40)

for filename, size in metadata_files['manifest_files']:
    file_path = os.path.join(metadata_path, filename)
    
    print(f"\n📄 Manifest: {filename}")
    
    try:
        manifest_df = spark.read.format("avro").load(f"file://{file_path}")
        
        # Delete files vs Data files 구분
        if manifest_df.count() > 0:
            # status 컬럼으로 추가/삭제 상태 확인
            status_summary = manifest_df.groupBy("status").count().collect()
            
            print(f"  📊 파일 상태별 분포:")
            for row in status_summary:
                status_text = "추가됨" if row['status'] == 1 else "삭제됨" if row['status'] == 2 else "기타"
                print(f"    {status_text}: {row['count']}개")
                
            # 샘플 데이터 확인
            print(f"  📋 샘플 엔트리:")
            manifest_df.select("status", "snapshot_id", "data_file").show(3, truncate=False)
            
    except Exception as e:
        print(f"  ❌ 분석 실패: {e}")

🔍 MOR 테이블의 Metadata Layer 분석
📊 MOR 테이블 메타데이터 현황:
  Metadata Files: 6개
  Manifest Files: 0개
  Manifest Lists: 5개
  Other Files: 25개

📋 최신 Metadata 파일 분석:
📄 파일: v6.metadata.json
📊 포맷 버전: 2
🆔 테이블 UUID: 92e8a123-b673-45c1-b5cc-175f2ce9bddf

📸 스냅샷 분석 (5개):
  1. ID: 3289915886533077444
     작업: unknown
     시간: 1755344560560
  2. ID: 5452519028519449692
     작업: unknown
     시간: 1755344562964
     🗑️ 추가된 Delete 파일: 16
  3. ID: 5991865128219017887
     작업: unknown
     시간: 1755344564040
     🗑️ 추가된 Delete 파일: 10
  4. ID: 3463369605987484001
     작업: unknown
     시간: 1755344564542
     🗑️ 추가된 Delete 파일: 3
  5. ID: 8152305441246547859
     작업: unknown
     시간: 1755344565104

📋 Manifest Files에서 Delete Files 추적:


In [15]:
# =============================================================================
# ⏰ MOR 테이블 Time Travel 실습
# =============================================================================

print("⏰ MOR 테이블 Time Travel 실습")
print("=" * 50)

# 1. 모든 스냅샷 확인
print("📸 모든 스냅샷 목록:")
snapshots_df = spark.sql("SELECT * FROM demo.blog.orders_mor.snapshots ORDER BY committed_at")
snapshots_df.show(truncate=False)

snapshots = snapshots_df.collect()

# 2. 각 스냅샷별 데이터 상태 확인
print(f"\n🕰️ 각 스냅샷별 데이터 상태:")
print("=" * 30)

for i, snapshot in enumerate(snapshots):
    snapshot_id = snapshot['snapshot_id']
    operation = snapshot['operation']
    committed_at = snapshot['committed_at']
    
    print(f"\n📸 스냅샷 #{i+1}:")
    print(f"  ID: {snapshot_id}")
    print(f"  작업: {operation}")
    print(f"  시간: {committed_at}")
    
    try:
        # 해당 스냅샷 시점의 데이터 조회
        snapshot_count = spark.sql(f"""
            SELECT COUNT(*) as count 
            FROM demo.blog.orders_mor 
            VERSION AS OF {snapshot_id}
        """).collect()[0]['count']
        
        print(f"  📊 레코드 수: {snapshot_count}")
        
        # 특정 스냅샷의 상위 5개 데이터
        if i < 3:  # 처음 3개 스냅샷만 상세 확인
            print(f"  📋 데이터 샘플:")
            spark.sql(f"""
                SELECT order_id, product_name, order_date, status 
                FROM demo.blog.orders_mor 
                VERSION AS OF {snapshot_id}
                ORDER BY order_id 
                LIMIT 5
            """).show()
            
    except Exception as e:
        print(f"  ❌ 조회 실패: {e}")

# 3. Time Travel 비교 분석
print(f"\n🔄 Time Travel 비교 분석:")
print("=" * 30)

if len(snapshots) >= 2:
    first_snapshot = snapshots[0]['snapshot_id']
    last_snapshot = snapshots[-1]['snapshot_id']
    
    print(f"📊 첫 번째 vs 마지막 스냅샷 비교:")
    
    # 첫 번째 스냅샷 데이터
    first_data = spark.sql(f"""
        SELECT COUNT(*) as count, SUM(amount) as total_amount
        FROM demo.blog.orders_mor 
        VERSION AS OF {first_snapshot}
    """).collect()[0]
    
    # 마지막 스냅샷 데이터  
    last_data = spark.sql(f"""
        SELECT COUNT(*) as count, SUM(amount) as total_amount
        FROM demo.blog.orders_mor
        VERSION AS OF {last_snapshot}
    """).collect()[0]
    
    print(f"  첫 번째 (ID: {first_snapshot}):")
    print(f"    레코드: {first_data['count']}개")
    print(f"    총액: ${first_data['total_amount']:,.2f}")
    
    print(f"  마지막 (ID: {last_snapshot}):")
    print(f"    레코드: {last_data['count']}개") 
    print(f"    총액: ${last_data['total_amount']:,.2f}")
    
    print(f"  변화:")
    print(f"    레코드: {last_data['count'] - first_data['count']:+d}개")
    print(f"    총액: ${(last_data['total_amount'] or 0) - (first_data['total_amount'] or 0):+,.2f}")

⏰ MOR 테이블 Time Travel 실습
📸 모든 스냅샷 목록:
+-----------------------+-------------------+-------------------+---------+------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                                 |summary                                                                                                                                                               